# Installations

In [ ]:
!pip install -q datasets transformers accelerate opendatasets evaluate fastervit

In [ ]:
!pip install timm==0.9.12

In [ ]:
import torch
if torch.cuda.is_available():
    device_name = torch.device("cuda")
else:
    device_name = torch.device('cpu')
print("Using {}.".format(device_name))

In [ ]:
!pip install huggingface_hub["fastai"]

# Imports

In [ ]:
from datasets import load_dataset, DatasetDict, load_metric, Dataset
from transformers import AutoImageProcessor, AutoFeatureExtractor, AutoModelForImageClassification, TrainingArguments, Trainer, ViTForImageClassification, ViTFeatureExtractor, ViTImageProcessor, Swinv2Model
from torchvision import transforms
from sklearn.metrics import accuracy_score, roc_curve, roc_auc_score
from sklearn.utils import resample
from sklearn.model_selection import train_test_split
from fastai.vision.all import *
from fastervit import create_model
from torchvision.io import read_image
from torchvision.io import ImageReadMode
from torchvision.transforms import (
    CenterCrop,
    Compose,
    Normalize,
    RandomHorizontalFlip,
    RandomResizedCrop,
    Resize,
    ToTensor,
)
import timm
import evaluate
import datasets
import numpy as np
import torch
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import opendatasets as od
import PIL

# Reusable Functions

In [ ]:
def convert_to_dataset(data):
    dataset = Dataset.from_pandas(data)
    return dataset

In [ ]:
def featureExtractor(model):
    print("Model :", model)
    return AutoFeatureExtractor.from_pretrained(model)

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)

    # Calculate accuracy
    accuracy = accuracy_score(labels, preds)

   # Calculate precision, recall, and F1-score
    precision = precision_score(labels, preds, average='weighted')
    recall = recall_score(labels, preds, average='weighted')
    f1 = f1_score(labels, preds, average='weighted')

    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1
    }

In [ ]:
def batch_sampler(examples):

    pixel_values = torch.stack([example["pixel_values"] for example in examples])
    for i in range(len(examples)):
      if examples[i]["pathology"] == 'MALIGNANT':
          examples[i]["pathology"] = 1
      elif examples[i]["pathology"] == 'BENIGN':
          examples[i]["pathology"] = 0
    labels = torch.tensor([example["pathology"] for example in examples])
    return {"pixel_values": pixel_values, "labels": labels}

In [ ]:
labels = ["Benign","Malignant"]
label2id, id2label = dict(), dict()
for i, label in enumerate(labels):
    label2id[label] = i
    id2label[i] = label

id2label[1]

In [ ]:
class ConvertToRGB(Transform):
    def __init__(self):
        pass

    def encodes(self, x: PIL.Image.Image) -> PIL.Image.Image:
        return x.convert('RGB')

# Import Clean Data from Hugging face

In [ ]:
dataset = load_dataset("Nicole-M/Dataset2")
dataset

In [ ]:
ds_train_devtest = dataset['test'].train_test_split(test_size=0.5, seed=42)

dataset = DatasetDict({
    'train': dataset['train'],
    'valid': ds_train_devtest['train'],
    'test': ds_train_devtest['test']
})
dataset

# Image Preprocessing

In [ ]:
# Transforms for pre-processing across a batch.
class imageTransform:
    columns = ["image file path", "cropped image file path"]

    def __init__(self, featureExtractor):
        size = (featureExtractor.size["height"], featureExtractor.size["width"])
        self.transforms = transforms.Compose([
            transforms.Resize(size),
            transforms.ToTensor(),
            transforms.RandomHorizontalFlip(p=0.5),
            transforms.RandomVerticalFlip(p=0.5),
            transforms.RandomRotation(degrees=180),
            # transforms.functional.to_pil_image(),
            transforms.Normalize(mean=featureExtractor.image_mean, std=featureExtractor.image_std),
        ])

    def __call__(self, example_batch):
        image_path = example_batch["image file path"][0]
        image = transforms.functional.to_pil_image(matplotlib.image.imread(image_path))
        example_batch["pixel_values"] = [self.transforms(image.convert("RGB")) for i in example_batch["image file path"]]
        return example_batch

In [ ]:
dataset["train"][9]

In [ ]:
# Transforms for pre-processing across a batch.
class valImageTransform:
    def __init__(self, featureExtractor):
        size = (featureExtractor.size["height"], featureExtractor.size["width"])
        self.transforms = transforms.Compose([
            transforms.Resize(size),
            transforms.ToTensor(),
            transforms.Normalize(mean=featureExtractor.image_mean, std=featureExtractor.image_std),
        ])

    def __call__(self, example_batch):
        image_path = example_batch["image file path"][0]
        image = transforms.functional.to_pil_image(matplotlib.image.imread(image_path))
        example_batch["pixel_values"] = [self.transforms(image.convert("RGB")) for i in example_batch["image file path"]]
        return example_batch

# VIT model

Load Model

In [ ]:
# VIT
VIT = "google/vit-base-patch16-224-in21k"

Setup

In [ ]:
ViTImageProcessor = ViTImageProcessor.from_pretrained(VIT)

In [ ]:
VitModel = ViTForImageClassification.from_pretrained(
    VIT,
    num_labels=2,
    label2id=label2id,
    id2label=id2label,
    ignore_mismatched_sizes = True, # provide this in case you're planning to fine-tune an already fine-tuned checkpoint
)

In [ ]:
# VitModel

In [ ]:
# VitModel.config

In [ ]:
# Set the training transforms
dataset["train"].set_transform(imageTransform(ViTImageProcessor))
# Set the validation transforms
dataset["valid"].set_transform(valImageTransform(ViTImageProcessor))
dataset["test"].set_transform(valImageTransform(ViTImageProcessor))

Train

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
args = TrainingArguments(
    remove_unused_columns=False,
    output_dir="./results/Vit-CBIS",
    eval_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=5,
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    logging_dir='./logs',
    weight_decay=0.01,
    lr_scheduler_type='linear',
    save_total_limit=1,
    # push_to_hub=True,
)

trainer = Trainer(
    model=VitModel,
    args=args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["valid"],
    tokenizer=ViTImageProcessor,
    compute_metrics=compute_metrics,
    data_collator=batch_sampler,
)

In [ ]:
# Fine-tune the model
vitTrainResult = trainer.train()

In [ ]:
kwargs = {
    "finetuned_from": "VIT",
    "tasks": "image-classification",
    "dataset": 'CBIS-DDSM',
    "tags": ['image-classification', 'breast cancer'],
}

trainer.push_to_hub('VIT-fineTuned', **kwargs)
trainer.create_model_card(**kwargs)


In [ ]:
vitTrainResult.metrics

In [ ]:
trainer.save_model()
trainer.log_metrics("train", vitTrainResult.metrics)
trainer.save_metrics("train", vitTrainResult.metrics)
trainer.save_state()

In [ ]:
# trainer.state.log_history

In [ ]:
history = pd.DataFrame(trainer.state.log_history)

In [ ]:
history.to_csv('Dataset2-ViT.csv')

In [ ]:
# # Evaluate the model
# eval_results = trainer.evaluate(val_data)
# trainer.log_metrics("eval", eval_results)
# trainer.save_metrics("eval", eval_results)
# print(f"Evaluation results: {eval_results}")

In [ ]:
from matplotlib import pyplot as plt
history.plot(kind='line', x='learning_rate', y='epoch')

In [ ]:
from matplotlib import pyplot as plt
history.plot(kind='line', x='learning_rate', y='loss')

In [ ]:
history.plot(kind='line', x='epoch', y='loss')

In [ ]:
from matplotlib import pyplot as plt
history.plot(kind='line', x='epoch', y='eval_accuracy')

In [ ]:
outputs = trainer.predict(dataset["test"])
print(outputs.metrics)

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

y_true = outputs.label_ids
y_pred = outputs.predictions.argmax(1)

labels = [0,1]
cm = confusion_matrix(y_true, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=labels)
disp.plot()

In [ ]:
# Assuming y_true are the true labels and y_scores are the predicted probabilities
y_true = outputs.label_ids
y_scores = outputs.predictions.argmax(1)

# Calculate the ROC curve
fpr, tpr, thresholds = roc_curve(y_true, y_scores)

# Calculate AUC
auc_score = roc_auc_score(y_true, y_scores)
print(f'AUC: {auc_score}')

# Plot the ROC curve
plt.figure()
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (AUC = {auc_score:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC)')
plt.legend(loc="lower right")
plt.show()

In [ ]:
from numpy import arange

# Load the training and validation loss dictionaries
train_loss = trainer.state.log_history

# Generate a sequence of integers to represent the epoch numbers
epochs = range(1, 21)

# Plot and label the training and validation loss values
plt.plot("loss", label='Training Loss')
plt.plot("eval_loss", label='Validation Loss')

# Add in a title and axes labels
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')

# Set the tick locations
plt.xticks(arange(0, 21, 2))

# Display the plot
plt.legend(loc='best')
plt.show()

# SWIN-V2 model

Load Model

In [ ]:
SwinV2 = "microsoft/swinv2-base-patch4-window8-256"

Setup

In [ ]:
SwinImageProcessor  = AutoImageProcessor.from_pretrained(SwinV2)

In [ ]:
SwinModel = AutoModelForImageClassification.from_pretrained(
    SwinV2,
    label2id=label2id,
    id2label=id2label,
    ignore_mismatched_sizes = True, # provide this in case you're planning to fine-tune an already fine-tuned checkpoint
)

In [ ]:
# SwinModel

In [ ]:
# SwinModel.config

In [ ]:
# Set the training transforms
dataset["train"].set_transform(imageTransform(SwinImageProcessor))
# Set the validation transforms
dataset["valid"].set_transform(valImageTransform(SwinImageProcessor))
dataset["test"].set_transform(valImageTransform(SwinImageProcessor))

Train

In [ ]:
training_args = TrainingArguments(
    remove_unused_columns=False,
    output_dir="./results/swinV2-CBIS",
    eval_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=5,
    weight_decay=0.01,
    logging_steps=10,
    logging_dir="./logs",
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    # push_to_hub=True,
    lr_scheduler_type='linear',
    save_total_limit=1,
)

# Instantiate the Trainer object
trainer = Trainer(
    model=SwinModel,
    args=training_args,
    data_collator=batch_sampler,
    compute_metrics=compute_metrics,
    train_dataset=dataset["train"],
    eval_dataset=dataset["valid"],
    tokenizer=SwinImageProcessor,
)

In [ ]:
# Fine-tune the model
swinTrainResult = trainer.train()

In [ ]:
kwargs = {
    "finetuned_from": "swinv2",
    "tasks": "image-classification",
    "dataset": 'CBIS-DDSM',
    "tags": ['image-classification', 'breast cancer'],
}

trainer.push_to_hub('SwinV2-fineTuned', **kwargs)
trainer.create_model_card(**kwargs)


In [ ]:
swinTrainResult.metrics

In [ ]:
trainer.save_model()
trainer.log_metrics("train", swinTrainResult.metrics)
trainer.save_metrics("train", swinTrainResult.metrics)
trainer.save_state()

In [ ]:
# trainer.state.log_history

In [ ]:
# # Evaluate the model
# eval_results = trainer.evaluate()
# print(f"Evaluation results: {eval_results}")

In [ ]:
history = pd.DataFrame(trainer.state.log_history)
history.head(5)

In [ ]:
history.to_csv('Dataset2-SwinV2.csv')

In [ ]:
# SwinMetrics = trainer.evaluate()
# trainer.log_metrics("eval", SwinMetrics)
# trainer.save_metrics("eval", SwinMetrics)

In [ ]:
# @title learning_rate vs epoch

from matplotlib import pyplot as plt
history.plot(kind='scatter', x='learning_rate', y='epoch', s=32, alpha=.8)
plt.gca().spines[['top', 'right',]].set_visible(False)

In [ ]:
# @title learning_rate vs epoch

from matplotlib import pyplot as plt
history.plot(kind='line', x='learning_rate', y='epoch')

In [ ]:
from matplotlib import pyplot as plt
history.plot(kind='line', x='learning_rate', y='loss')

In [ ]:
from matplotlib import pyplot as plt
history.plot(kind='line', x='epoch', y='loss')

In [ ]:
# @title accuracy vs epoch

from matplotlib import pyplot as plt
history.plot(kind='line', x='loss', y='step')

In [ ]:
swinOutputs = trainer.predict(dataset["test"])
print(swinOutputs.metrics)

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

y_true = swinOutputs.label_ids
y_pred = swinOutputs.predictions.argmax(1)

labels = [0,1]
cm = confusion_matrix(y_true, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=labels)
disp.plot()

In [ ]:
# Assuming y_true are the true labels and y_scores are the predicted probabilities
y_true = swinOutputs.label_ids
y_scores = swinOutputs.predictions.argmax(1)

# Calculate the ROC curve
fpr, tpr, thresholds = roc_curve(y_true, y_scores)

# Calculate AUC
auc_score = roc_auc_score(y_true, y_scores)
print(f'AUC: {auc_score}')

# Plot the ROC curve
plt.figure()
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (AUC = {auc_score:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC)')
plt.legend(loc="lower right")
plt.show()

# Import from CSV

In [ ]:
data = od.download("https://www.kaggle.com/datasets/awsaf49/cbis-ddsm-breast-cancer-image-dataset")

* Rename this downloaded dataset to "dataset" for FastVit to access images

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Data/cbisDdsm-Train.csv', header=None, index_col=False)

In [ ]:
df[0].replace('MALIGNANT', 1, inplace=True)
df[0].replace('BENIGN', 0, inplace=True)
df.head(5)

In [ ]:
dfTest = df.tail(300)
dfTest.shape

In [ ]:
dfTest.head(15)

In [ ]:
df.drop(df.tail(300).index,inplace = True)
df.shape

# Fast VIT model

Load Model

In [ ]:
# Fast VIT
pretrained_fastVit = timm.create_model('fastvit_sa24.apple_in1k', pretrained=True, num_classes=2)

In [ ]:
# pretrained_fastVit

In [ ]:
item_tfms = [Resize((224,224))]
            #  transforms.ToTensor()]
# , ConvertToRGB(), transforms.ToPILImage(), Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
batch_tfms = [Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])]

In [ ]:
dls = ImageDataLoaders.from_df(path='/', df=df, valid_pct=0.2, label_col=0, fn_col=1, item_tfms=item_tfms, bs=16)
                              #   batch_tfms=batch_tfms)

In [ ]:
from fastai.vision.all import *

learner = vision_learner(dls, 'fastvit_sa24.apple_in1k', metrics=accuracy)
learner.lr_find()

In [ ]:
learner.fine_tune(5, 1e-5)

In [ ]:
learner.recorder.plot_loss()

In [ ]:
results = ClassificationInterpretation.from_learner(learner)
results.plot_confusion_matrix()

In [ ]:
results.most_confused(min_val=50)

In [ ]:
results.plot_top_losses(9, figsize= (16,16))

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
from huggingface_hub import push_to_hub_fastai

repo_id = "Nicole-M/fastViT-CBIS"

push_to_hub_fastai(learner=learner, repo_id=repo_id)

In [ ]:
learner.save('/content/CBIS_DDSM_FastViT')
learner.export()

In [ ]:
!cp -r '/content/CBIS_DDSM_FastViT.pth' /content/drive/MyDrive/Data/

# Faster VIT model

Load Model

In [ ]:
# Load FasterViT model
model = create_model('faster_vit_3_224', pretrained=True, model_path="/content/drive/MyDrive/Pretrained-models/fastervit_3_224_1k.pth.tar")

# Print the model architecture
# print(model)

In [ ]:
# Modify the final layer for custom classification
num_ftrs = model.head.in_features
model.head = torch.nn.Linear(num_ftrs, 2)

In [ ]:
# Move the model to GPU if available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [ ]:
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import DataLoader

# Define loss function
criterion = torch.nn.CrossEntropyLoss()

# Define optimizer
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Learning rate scheduler
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, dataframe):
        self.dataframe = pd.DataFrame(dataframe)

    def __getitem__(self, index):
        # path = "/content/dataset/"
        imagePath = self.dataframe.iloc[index, 1]
        label = self.dataframe.iloc[index, 0]
        # print('/'+imagePath)
        image = read_image('/'+ imagePath, mode=ImageReadMode.RGB)
        image = transforms.Resize((224,224))(image)
        image = transforms.RandomRotation(180)(image)
        return image, label

    def __len__(self):
        return len(self.dataframe)

data = CustomDataset(dataframe=df)
trainDataloader = DataLoader(data, 16, shuffle=True )
# for sample in trainDataloader:
#     print(sample)
#     break

In [ ]:
class CustomTestDataset(Dataset):
    def __init__(self, dataframe):
        self.dataframe = pd.DataFrame(dataframe)
        # print(self.dataframe)

    def __getitem__(self, index):
        path = "/"
        imagePath = self.dataframe.iloc[index, 1]
        label = self.dataframe.iloc[index, 0]
        image = read_image(path+imagePath, mode=ImageReadMode.RGB )
        image = transforms.Resize((224,224))(image)
        return image, label

    def __len__(self):
        return len(self.dataframe)

data = CustomTestDataset(dataframe=dfTest)
testDataloader = DataLoader(data, 16, shuffle=True)
# for sample in testDataloader:
#     print(sample)
#     break

In [ ]:
class_names = ['0', '1']

In [ ]:
# Training loop
num_epochs = 6
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

train_losses = []  # To store the losses for plotting
best_val_loss = float('inf')  # Initialize with a very large value
prediction_list = []
label_list = []
y_pred = []
y_true = []

# Train the model
for epoch in range(num_epochs):

    # Train the model on the training set
    model.train()

    # Initialize the training loss accumulator to zero
    training_loss = 0.0

    for i, (image, labels) in enumerate(trainDataloader):
        # Prepare data and send it to the proper device
        image = image.to(device)
        labels = labels.float().to(device)

        # Clear the gradients of all optimized parameters
        optimizer.zero_grad()

        # Forward pass: obtain model predictions for the input data
        outputs = model(image.float())

        # Compute the loss between the model predictions and the true labels
        loss = criterion(outputs, labels.long())

        # Backward pass: compute gradients of the loss with respect to model parameters
        loss.backward()

        # Update model parameters using the computed gradients and the optimizer
        optimizer.step()

        # Update the training loss
        training_loss += loss.item()

    # Calculate average training loss
    train_loss = training_loss / len(trainDataloader)
    train_losses.append(train_loss)

# Evaluate the model on the validation set
    model.eval()
    val_loss = 0.0
    correct_preds = 0
    total_samples = 0
    with torch.no_grad():
        for image, labels in testDataloader:
            # Prepare data and send it to the proper device
            image = image.to(device)
            labels = labels.float().to(device)

            # Forward pass: obtain model predictions for the input data
            outputs = model(image.float())

            # Compute the loss between the model predictions and the true labels
            loss = criterion(outputs, labels.long())

            # Update the validation loss
            val_loss += loss.item()

             # Round up and down to either 1 or 0
            predicted = torch.round(outputs)
            total_samples += labels.size(0)

            # Calculate how many images were correctly classified
            correct_preds += torch.sum(torch.all(torch.eq(predicted, torch.argmax(labels.cpu(), dim=0)), dim=1)).item()

            # Gather all predictions
            prediction_list.extend(predicted.cpu())
            label_list.extend(labels.cpu())


    output2 = (torch.max(torch.exp(predicted), 1)[1]).data.cpu().numpy()
    y_pred.extend(output2) # Save Prediction

    labels2 = labels.data.cpu().numpy()
    y_true.extend(labels2) # Save Truth


    # Calculate validation loss
    val_loss /= len(testDataloader)

    # Calculate validation accuracy
    val_acc = correct_preds / total_samples * 100

    # Print validation loss and accuracy
    print(f"Epoch [{epoch + 1}/{num_epochs}] Train Loss: {train_loss:.4f}  Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_acc:.2f}%")

    # Save the model if it performs better on validation set
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), f'/content/best_model_epoch_{epoch + 1}.pth')

print('Finished Training')

# Plotting the evolution of loss
plt.plot(train_losses, label='Training Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Evolution of Training Loss')
plt.legend()
plt.show()

In [ ]:
# Save the model
torch.save(model.state_dict(), 'fasterVit_model1.pth')

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

cm = confusion_matrix(y_true, y_pred)
ConfusionMatrixDisplay(cm).plot()

# Save to Google Drive

In [ ]:
!cp -r '/content/faster_vit_custom_model.pth' /content/drive/MyDrive/Data/